<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Csatoljuk a Google Drive-ot
from google.colab import drive
drive.mount('/content/drive')
!pip install Pillow

import os
from PIL import PngImagePlugin, Image

# Állítsd be a kép útvonalát (ha Drive-on van, a /content/drive/... szerint)
image_path = '/content/drive/MyDrive/Image/Banner_00001_.png'  # például

# Megnyitjuk a képet
with Image.open(image_path) as img:
    # A Pillow PngImagePlugin-je segítségével beolvassuk a text chunk-okat
    if isinstance(img, PngImagePlugin.PngImageFile):
        metadata = img.text
        if metadata:
            print("Metaadatok kulcsai és értékei:\n")
            for k, v in metadata.items():
                print(f"**{k}**:\n{v}\n")
        else:
            print("Nem találtunk metaadatokat a PNG text chunk-okban.")
    else:
        print("Ez a fájl nem PNG, vagy nem PngImageFile típusú.")
